In [1]:
import pandas as pd

data_directory = '../data/'

# here is the corrected data
filename = "corrected_data.json"

# take the transpose so that the column names are the metadata keys, not the story ids
df = pd.read_json(data_directory + filename).transpose()

# let's see what it looks like!
df.head()

,author_id,characters,genres,language,num_chapters,num_favs,num_follows,num_reviews,num_words,published,rated,status,title,updated
10000036,2846408,"[[Ron W., Hermione G.]]","[Romance, Humor]",English,1,5,2,3,5436,1389050085,T,Complete,Kilts and other adversities,1389050085
10000109,5232542,"[James P., Lily Evans P.]","[Humor, Romance]",English,1,4,NaN,2,747,1389051817,K+,Complete,Of Dead Puppies and Dropped Pianos,1389051817
10000111,5438139,[],[],Spanish,1,7,6,5,400,1389051884,K+,Incomplete,Querido primo Harry,1389051884
10000114,5437478,"[Hermione G., Draco M.]","[Romance, Humor]",Spanish,1,11,13,8,1757,1389051911,K+,Incomplete,Conciertos en Hogwarts,1389051911
10000137,4626918,"[Bellatrix L., Luna L.]","[Humor, Horror]",English,3,NaN,NaN,3,659,1389052244,T,Complete,Crazy songs from crazy people,1389414254


In [2]:
sorted_df = df.sort_values(['published'], ascending = True)
sorted_df.head()

,author_id,characters,genres,language,num_chapters,num_favs,num_follows,num_reviews,num_words,published,rated,status,title,updated
1657629,509449,[Harry P.],[Adventure],English,29,4491,3397,4699,470346,0,T,Incomplete,Harry Potter and the Power of Time,1177495284
3803,5402,[],[],English,4,155,36,587,13605,936428400,K+,Incomplete,Harry Potter and the Man of Unknown,936428400
6826,3986,[],[],English,1,20,14,179,2731,938415600,K+,Incomplete,Hermione & Harry,938415600
7588,5402,[],[],English,2,76,10,396,7782,939798000,K+,Incomplete,The Love That Shattered a Man,939798000
684,4069,"[Lily Evans P., James P.]","[Romance, Angst]",English,1,83,12,363,10649,940489200,K+,Incomplete,Lily,940489200


In [3]:
sorted_df.loc[sorted_df.published == 0, 'published'] = 1177495284

In [4]:
characters_only = sorted_df[sorted_df['characters'].apply(lambda x: len(x)>0)]
len(characters_only)

507475

Okay, that's all the data. Time to tease out some characters... I think it'd be good to have:
Harry P.
Ron W.
Hermione G.
Sirius B.
Remus L.
Albus D.
Albus S. P.
Draco M.
Severus S.
Newt S.
Neville L.
Gellert G.
N. Tonks
Bellatrix L.

In [45]:
min_df = characters_only[['characters','published']]
min_df.head()

,characters,published
1657629,[Harry P.],1177495284
684,"[Lily Evans P., James P.]",940489200
3476,"[Harry P., Lily Evans P.]",942739200
7795,"[Harry P., Hermione G.]",945590400
5111,"[Ginny W., Ron W.]",946886400


In [46]:
min_df.loc[:,'published'] = min_df['published'].map(lambda x: convertToDateTimeObj(x))
min_df.head()

/Users/allisonking/anaconda2/envs/fanfic/lib/python3.6/site-packages/pandas/core/indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,characters,published
1657629,[Harry P.],2007-04-25 10:01:24
684,"[Lily Evans P., James P.]",1999-10-21 07:00:00
3476,"[Harry P., Lily Evans P.]",1999-11-16 08:00:00
7795,"[Harry P., Hermione G.]",1999-12-19 08:00:00
5111,"[Ginny W., Ron W.]",2000-01-03 08:00:00


In [38]:
import datetime
def convertToDateTimeObj(timestamp):
    return datetime.datetime.utcfromtimestamp(timestamp)

In [9]:
character_mapping = {'Harry P.' : 'Harry Potter', 'Ron W.' : 'Ron Weasley', 'Hermione G.' : 'Hermione Granger',
                     'Draco M.' : 'Draco Malfoy', 'Neville L.' : 'Neville Longbottom',
                    'Sirius B.' : 'Sirius Black', 'Remus L.' : 'Remus Lupin', 'N. Tonks' : 'Nymphadora Tonks',
                     'Bellatrix L.' : 'Bellatrix Lestrange', 'Severus S.' : 'Severus Snape', 
                     'Albus D.' : 'Albus Dumbledore', 'Gellert G.' : 'Gellert Grindelwald', 'Newt S.' : 'Newt Scamander',
                     'Albus S. P.' : 'Albus Severus Potter'
                    }

In [47]:
test = min_df[min_df['characters'].apply(lambda x: 'Harry P.' in x)]


In [48]:
len(test)

144067

In [66]:
group_test = test.groupby(pd.DatetimeIndex(test['published']).date).published.count()
len(group_test)

5978

In [67]:
group_df = group_test.to_frame().reset_index()
group_df.head()

,index,published
0,1999-11-16,1
1,1999-12-19,1
2,2000-02-02,1
3,2000-02-22,1
4,2000-02-26,1


In [71]:
group_df.columns=['published','count','name']
group_df.head()

,published,count,name
0,1999-11-16,1,Harry P.
1,1999-12-19,1,Harry P.
2,2000-02-02,1,Harry P.
3,2000-02-22,1,Harry P.
4,2000-02-26,1,Harry P.


In [68]:
group_df['name'] = 'Harry P.'

In [86]:
def getCharacterDf(character_name):
    character = min_df[min_df['characters'].apply(lambda x: character_name in x)]
    grouped = character.groupby(pd.DatetimeIndex(character['published']).date).published.count()
    grouped_df = grouped.to_frame().reset_index()
    grouped_df['name'] = character_mapping[character_name]
    grouped_df.columns=['published', 'count', 'name']
    return grouped_df

In [80]:
harry = getCharacterDf('Harry P.')
print(len(harry))
harry.head()

5978


,"published,",count,name
0,1999-11-16,1,Harry Potter
1,1999-12-19,1,Harry Potter
2,2000-02-02,1,Harry Potter
3,2000-02-22,1,Harry Potter
4,2000-02-26,1,Harry Potter


In [78]:
min_df.head()

,characters,published
1657629,[Harry P.],2007-04-25 10:01:24
684,"[Lily Evans P., James P.]",1999-10-21 07:00:00
3476,"[Harry P., Lily Evans P.]",1999-11-16 08:00:00
7795,"[Harry P., Hermione G.]",1999-12-19 08:00:00
5111,"[Ginny W., Ron W.]",2000-01-03 08:00:00


In [81]:
character_df_array = []
for name in list(character_mapping.keys()):
    result = getCharacterDf(name)
    character_df_array.append(result)

In [82]:
all_character_df = pd.concat(character_df_array)
len(all_character_df)

59418

In [83]:
all_character_df.head()

,"published,",count,name
0,1999-11-16,1,Harry Potter
1,1999-12-19,1,Harry Potter
2,2000-02-02,1,Harry Potter
3,2000-02-22,1,Harry Potter
4,2000-02-26,1,Harry Potter


In [84]:
all_character_df.tail()

,"published,",count,name
2792,2017-07-20,2,Albus Severus Potter
2793,2017-07-23,2,Albus Severus Potter
2794,2017-07-27,1,Albus Severus Potter
2795,2017-07-28,4,Albus Severus Potter
2796,2017-07-29,1,Albus Severus Potter


In [85]:
all_character_df.to_csv('../data/char-pub-grouped.csv', index=False)